In [1]:
import math
from tensorboardX import SummaryWriter
import roboschool
import quadruppedEnv
import env_configurations
sess = None

robot lib not fouund

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers

env_name = 'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
env = env_configurations.a2c_configurations[env_name]['ENV_CREATOR']()
print(env.action_space.low)
print(env.action_space.high)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-05-26 19:49:20,265	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-26_19-49-20_12138/logs.
2019-05-26 19:49:20,378	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:44765 to respond...
2019-05-26 19:49:20,528	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:46558 to respond...
2019-05-26 19:49:20,536	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-05-26 19:49:20,596	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-26_19-49-20_12138/logs.
2019-05-26 19:49:20,598	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.104',
 'redis_address': '192.168.3.104:44765',
 'object_store_address': '/tmp/ray/session_2019-05-26_19-49-20_12138/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-05-26_19-49-20_12138/sockets/raylet',
 'webui_url': None}

In [ ]:
from a2c_v2 import A2CAgent
import tr_helpers
import networks
import a2c_games_configurations

a2c_config = a2c_games_configurations.quadrupped_config
#a2c_config = a2c_games_configurations.roboschoolant_config
#self, sess, env_name, observation_shape, actions_num, config = default_config
observation_shape = env.observation_space.shape
action_space = env.action_space
print(observation_shape)
agent = A2CAgent(sess,'rob_20', env.observation_space, False, action_space, a2c_config)
#agent.restore('nn/ppo_continuousQuadruppedWalk-v1')
agent.train()


(84,)
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


2019-05-26 19:49:21,820	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=12225) 
(pid=12225) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=12225) For more information, please see:
(pid=12225)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=12225)   * https://github.com/tensorflow/addons
(pid=12225) If you depend on functionality not listed there, please file an issue.
(pid=12225) 
(pid=12225) robot lib not fouund
(pid=12225) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=12225) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=12225) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=12225)   result = entry_point.load(False)
(pid=12229) 
(pid=12229) WARNING: The TensorFlow contrib module will not be included in Tens

(pid=12228) 
(pid=12228) WARNING: The TensorFlow contrib module will not be included in TensorFlow 2.0.
(pid=12228) For more information, please see:
(pid=12228)   * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
(pid=12228)   * https://github.com/tensorflow/addons
(pid=12228) If you depend on functionality not listed there, please file an issue.
(pid=12228) 
(pid=12224) robot lib not fouund
(pid=12224) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=12224) WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
(pid=12224) /home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
(pid=12224)   result = entry_point.load(False)
(pid=12252) robot lib not fouund
(pid=12252) WARN: gym.spaces.Box autodetected dtype as <cla

Frames per seconds:  841.8352571718491
saving next best rewards:  -157.72958131134726
Frames per seconds:  959.5794565767441
saving next best rewards:  -155.26639127949986
Frames per seconds:  939.5726161171292
saving next best rewards:  -154.33814698680067
Frames per seconds:  840.0169307239412
Frames per seconds:  958.4186075209656
saving next best rewards:  -153.4453806613952
Frames per seconds:  908.9230087146699
saving next best rewards:  -149.4224891600929
Frames per seconds:  936.7654607691579
Frames per seconds:  857.3447851781025
saving next best rewards:  -148.75810146894105
Frames per seconds:  887.1508280464491
saving next best rewards:  -147.18055173112617
Frames per seconds:  827.3191723913689
saving next best rewards:  -146.6242644319059
Frames per seconds:  768.6433531216509
saving next best rewards:  -143.74442094117106
Frames per seconds:  818.1713323821583
saving next best rewards:  -143.2833361403954
Frames per seconds:  899.3797469865314
saving next best rewards:  

In [ ]:
ray.shutdown()

In [ ]:
import numpy as np
#agent.save('nn/latest')
#agent.restore('nn/rob_20QuadruppedWalk-v1')
#agent.restore('nn/ppo_continuousQuadruppedWalk-v1')

def evaluate(env,t_max=5000):
    rewards = []
    env._max_episode_steps = 5000
    env.batch_mode = False
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(5000):
        action = agent.get_action([s], False)
        
        #ad = agent.get_action_distribution([s])
        #print(ad)
        s, r, done, _ = env.step(np.squeeze(action, axis = 0))
        print(action)
        env.render()
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers

#env_monitor = wrappers.make_atari_deepmind(env_name, noop_max=30, skip=4)
#env_monitor = wrappers.ReallyDoneWrapper(env_monitor)
#env_monitor = gym.wrappers.Monitor(env,directory='video_ppo',force=True)

sessions = [print('reward:', evaluate(env)) for _ in range(1)]
#env_monitor.close()




In [ ]:
obs = env.reset()
obs, r, done, _ = env.step(2)

done = False
for _ in range(130):
    obs, _, done, _ = env.step(1)
    #env.render()
    obs = np.array(obs)
    if done:
        print('done')
        break
plt.title("what your network gonna see")
print (obs.shape)
plt.imshow(obs[:,:,0],interpolation='none',cmap='gray');

In [ ]:
import gym
gym.envs.registry.all()